# Example of user environment configuration

If you are a Linux user, you might sometime want to create your own executable (for example, a little bash script checking if your servers are alive, or a Python script checking the network usage...). Obviously, some of these scripts can not be installed globally on your system.

If you ask a student in my engineering school how to deal with it, about 90% of them will answer the following : *just put an alias in your `.whateverrc` like this :*

```bash
alias mycommand='/home/user/some/folders/myscript.sh'
```

Okay, it works. But when you have more than 10 scripts, more than only one shell, environment dependant scripts etc, it begins to be a bit hard to manage all your scripts with just aliases.

The overall idea of this post is to show you **a way** (not the only way) to manage your user environment, in order to simplify these kinds of things.


# Unix philosophy : make your life easier

So just to explain, I use 2 different shells (`zsh` about everytime, and sometimes `bash` for really specific operations). I have something like 30 different scripts, 10 crons and stuff like that. Managing everything with aliases is just impossible. I had to find a cool setup, allowing me to simplify my life.

The point of my configuration is to purpose a setup for managing :

- global environment definition
- shell dependant environment (like prompt, shell configuration...)
- script startup without using alias
- and some stuff like this...

This setup depends of just few things and is pretty simple :

- use a `.profile` to manage your global environment seting
- use your `.whateverrc` file to manage your shells
- create a `bin/` directory containing all your scripts (or symbolic link to them)
- and... that's it ? :)

## `.profile`, the global seting

The `.profile` file handle all your global configuration. It's here you define your global aliases, your global environment configuration and whatever you think useful.

Here is part of mine (it is way longer) :

```sh
#!/bin/sh

# Aliases
########################################

# Listings
alias ls="ls --color=auto"
alias ll="ls -larth"
alias sl="ls"   # Mistakes were made :)
alias la="ls -la"

# Python aliases
alias notebook="cd ~/NoteBooks && ipython notebook"

# Environment
########################################

export LANG=fr_FR.UTF-8
export PATH=$HOME/bin:$PATH   # Add the bin directory for path finding
export EDITOR=vim             # Set the editor (useful for git)
export TERM="xterm-256color"  # Terminal setup for supporting 256 colors

# My scripts setup
########################################

# git-watcher cron & script
export GIT_WATCHER_CONFIG_FILE="$HOME/.config/gitwatcher/datas.txt"
export GIT_WATCHER_NOTIFY_ICON="$HOME/.config/gitwatcher/icon.png"

# DBUS exportation
export DBUS_VALUE_FILEPATH="$HOME/.dbus/Xdbus"

# Automations
########################################

# DBUS environment variable exportation
if [ ! -s "$DBUS_SESSION_BUS_ADDRESS" ]; then
    env [ grep DBUS_SESSION_BUS_ADDRESS | sed "s/^/export /g" > $DBUS_VALUE_FILEPATH
fi

if [[ -f $DBUS_VALUE_FILEPATH ]]; then
    source $DBUS_VALUE_FILEPATH
fi
```

Let's go through this file.

- Aliases are some shared aliases between my shells. In my mind, it is just some common things I use everytime. You **really** should not make aliases to content modifier commands (like `cp`, `rm`, `mv`...). These are shell dependents, and this is exactly the reason I use two different shells.
- After that, I define my environment setings, shared between all my shells. The most important one is `$PATH`. Defining this command just tell your shell to find commands into a new source directory. The value might look like `/usr/local/sbin:/usr/local/bin:/usr/bin` (or longer).
- My scripts often share some values between them. Instead of defining somewhere else I will forget 2 days after, I define my script setup here.
- And to finish, some little automations that will be run every time I open a shell / subshell.

## Why should I use my shell rc ?

Because it's the configuration of your shell. Not your whole environment configuration. The reason I use 2 shell is pretty easy to understand. I use about 99% of the time `zsh`, because it's cool, simple and it has a lot of modules. However, I use another shell to do some dangerous operations (like removing or moving large set of data...). Because of this, my prompts are different, my aliases are different, a lot of things are different.

Here is a part of my `.zshrc` :

```bash
#!/bin/zsh

# Configuration Option ('cause it's better with options)
########################################

setopt prompt_subst        # Enable dynamic prompting

# God damn, let's load some constants
########################################

source $HOME/.profile           # Global profile
source $HOME/.zsh_aliases       # Source my aliases
eval "`dircolors -b $HOME/.ls_colors`" # Colors for ls, tree...

# Okay, now i want zsh do all the things for me.
########################################

autoload colors && colors  # Guess what ? colors.
setopt correctall          # When you do shit, i'll correct it

# Completion configuration
zstyle ':completion:*' list-colors "${(@s.:.)LS_COLORS}"
autoload -Uz compinit
compinit
zstyle ':completion:*:descriptions' format '%U%B%d%b%U'
zstyle ':completion:*:warnings' format '%BSorry, no matches for: %d%b'

# Best prompt in da world !
########################################

# This is a git formatter for side informations
function gitformat() {
    if git rev-parse 2> /dev/null; then
        local branch=${$(git symbolic-ref HEAD 2> /dev/null)#refs/heads/}
        local hashcode=$(git rev-parse --short HEAD 2> /dev/null)
        local nb_untracked=$(git status --porcelain | grep "^??" | wc -l)
        local nb_modified=$(git status --porcelain | grep -v "^??" | wc -l)
        if git diff-index --quiet HEAD --; then
            local color="blue"
        else
            local color="red"
        fi

        echo -n "%{$fg_bold[${color}]%}${branch} %{$reset_color%}[%{$fg[${color}]%}${hashcode}%{$reset_color%}] "
        echo "%{$fg_bold[${color}]%}${nb_modified}%{$reset_color%}|%{$fg_bold[cyan]%}${nb_untracked}%{$reset_color%} -"
    fi
}

preexec () {
    echo -ne "" # reset scrolling region
}

function _formatuser() {
    if [ "$UID" != "0" ]; then
        echo -n "%{$fg_bold[yellow]%}"
    else
        echo -n "%{$fg_bold[red]%}"
    fi

    echo -n "$USERNAME%{$reset_color%}@%{$fg_bold[cyan]%}$HOST"
}

function _formatevent() {
    echo -n "%{$fg_bold[green]%}%i%{$reset_color%}%{$reset_color%}"
}

function _formatpath() {
    echo -n "%{$fg_bold[white]%}%~%{$reset_color%}"
}

PROMPT='
%{$fg_bold[magenta]%}%* %b%{$fg[magenta]%}(%D)%{$reset_color%} $(gitformat) 
┌─[$(_formatuser) $(_formatevent)] $(_formatpath)
└─ %# '
```

And this is **a part** of it. This is my common shell. I use it whenever I want, whenever I need etc. However, when I'm manipulating data that might be dangerous (like removing stuff etc), I prefere using `bash`. My bashrc is really clean :

```bash
#!/bin/bash

# Source the profile
source $HOME/.profile

# Just a better prompt
PS1='${debian_chroot:+($debian_chroot)}\[\033[01;32m\]\u\[\033[00m\]@\[\033[01;31m\]funkysayu\[\033[00m\]:\[\033[01;34m\]\w\[\033[00m\]\n->\$ '

# Default color for ls and tree
export LS_OPTIONS='--color=auto'
eval "`dircolors`"

# Some more alias to avoid making mistakes:
alias rm='rm -i'
alias cp='cp -i'
alias mv='mv -i'
```

Simple, clean, don't need more.

## Put your scripts in your `bin/` directory

As said before, we extended the variable `$PATH` in the `.profile`. That means that every shell have this variable extended, and you can now put your scripts into it and call them with a simple command.

Let's take for example the following scripts :
```bash
#!/bin/bash

echo "Hey, I'm a tomato !"
```

If you put this script into the file located to `$HOME/bin/tomato` and setup the execution right on it (`chmod +x $HOME/bin/tomato`), you can then execute this as a command :

```
$ tomato
Hey, I'm a tomato !
```

**Note**: your completion may not find the new `tomato` command, because your shell may not update everytime your commands list. To update your completion database, you can restart your shell.

# Example : `git-watcher`

At this point, you may think that it's using the bazooka to shoot the ant. Let's take a cool example : using `notify-send` into a `cron`. It's the kind of thing I love doing. For example, everytime I have a new mail, I receive a notification from one of my script on my desktop. This is a really cool stuff.

We will take as example my `git-watcher` module. This module checks every 5 minutes if a new push has been done on a git I'm "watching".

I use two scripts :

- the `watch-git` script to add a repository to watch
- the `git-watcher` script that will send notifications through a cron.

These two scripts use shared environment variables :

- `GIT_WATCHER_CONFIG_FILE` containing all the repositorys to watch,
- `GIT_WATCHER_NOTIFY_ICON` which is the icon used for notifications,
- `DBUS_SESSION_BUS_ADDRESS` indicating which interface to send notifications on (required by `notify-send`).

## Define the bash script `watch-git` to add a repository in the watch list

First off, I created a bash script into my `bin/` directory named `watch-git`. This script is pretty simple :

```bash
#!/bin/bash

# Just checking if user knows how the script works
if [[ $# -ne 2 ]] ; then
    echo "Usage : $0 {repository} {branch}"
    echo "Add to the git-watcher cron a repository to watch."
    exit 1
fi

# Checking if environment variables are set
if [ -z $GIT_WATCHER_CONFIG_FILE ]; then
    echo "Error: GIT_WATCHER_CONFIG_FILE environment variable not set."
    echo "Set it to know where I should put my datas !"
    exit 1
fi

# Creating file if not exists
if [ ! -f "$GIT_WATCHER_CONFIG_FILE" ] ; then
    echo "Git watcher config file not found. Creating a new one."
    if [ ! -d `dirname $GIT_WATCHER_CONFIG_FILE` ]; then
        mkdir -p `dirname $GIT_WATCHER_CONFIG_FILE`
    fi
    touch "$GIT_WATCHER_CONFIG_FILE"
fi

repository="$1"
branch="$2"

# Check if not already watching...
grep "$repository $branch" "$GIT_WATCHER_CONFIG_FILE" && \
     echo "You are already watching this repository." && \
     exit 1

hashcode="0000000000000000000000000000000000000000"
echo -e "$hashcode\t$repository\t$branch" >> $GIT_WATCHER_CONFIG_FILE
```

With this, you have now a new command `watch-git` allowing you to add a repository to watch.

## Define the cron bash script `git-watcher` to send you notifications every 5 minutes

Let's take a look to the cron script. He is a bit long, but the sense is pretty simple to get.

```bash
#!/bin/bash

# Load environment variables if they exists. Else exit 1
if [ -f "$HOME/.profile" ]; then
    . "$HOME/.profile"
else
    echo "Profile not found. Exiting."
    exit 1
fi

# Creating git watcher config file if it doesn't exists
if [ ! -f $GIT_WATCHER_CONFIG_FILE ] ; then
    echo "Git watcher config file not found. Creating an empty one."
    if [ ! -d $(dirname $GIT_WATCHER_CONFIG_FILE) ]; then
        mkdir -p $(dirname $GIT_WATCHER_CONFIG_FILE)
    fi
    touch $GIT_WATCHER_CONFIG_FILE
fi

# Creating temp dir to fetch git watched and git results
new_config=`mktemp`

# Actually do the shit
while read line
do
    # Parsing basic fields
    offlinehash=`echo $line | cut -d " " -f 1`
    repository=`echo $line | cut -d " " -f 2`
    branch=`echo $line | cut -d " " -f 3`

    # Parsing more specific fields
    server=${repository#*:}
    reponame=${repository%:*}

    # Getting hashes
    onlinehash=`git ls-remote $repository $branch`
    if [ "$?" -ne "0" ] ; then
        echo "Failure : unable to retrieve informations."
        rm "$new_config"
        exit 1
    fi
    onlinehash=`echo $onlinehash | cut -d ' ' -f 1`

    # Send notifications if there's a new push
    if [ "$offlinehash" != "$onlinehash" ] ; then
        notify-send "GitWatcher : new push detected" \
            "Repository: $reponame\nServer: $server" \
            -i "$GIT_WATCHER_NOTIFY_ICON"
    fi

    echo -e "$onlinehash\t$repository\t$branch" >> "$new_config"
done < "$GIT_WATCHER_CONFIG_FILE"

# Move the new configuration
mv "$new_config" "$GIT_WATCHER_CONFIG_FILE"
```

## Setup your environment

As said before, you need 3 variables (`GIT_WATCHER_CONFIG_FILE`, `GIT_WATCHER_NOTIFY_ICON` and `DBUS_SESSION_BUS_ADDRESS`). These are *shared* variables, because they can be used in any shells. So we define this variables into the `.profile` file :

```bash
# .../...

# My scripts setup
########################################

# .../...

# git-watcher cron & script
export GIT_WATCHER_CONFIG_FILE="$HOME/.config/gitwatcher/datas.txt"
export GIT_WATCHER_NOTIFY_ICON="$HOME/.config/gitwatcher/icon.png"

# DBUS exportation
export DBUS_VALUE_FILEPATH="$HOME/.dbus/Xdbus"

# Automations
########################################

# DBUS environment variable exportation
if [ ! -s "$DBUS_SESSION_BUS_ADDRESS" ]; then
    env [ grep DBUS_SESSION_BUS_ADDRESS | sed "s/^/export /g" > $DBUS_VALUE_FILEPATH
fi

if [[ -f $DBUS_VALUE_FILEPATH ]]; then
    source $DBUS_VALUE_FILEPATH
fi

# .../...
```

## Add your cron to your crontab

The last step is to add your `git-watcher` into your crontab. Actually doing this is pretty simple. Just run a `crontab -e` and add the following line :

```
*/5 * * * * $HOME/bin/cron/git-watcher
```

If you did not understand the above line, you should take a look to <a href="http://www.cyberciti.biz/faq/how-do-i-add-jobs-to-cron-under-linux-or-unix-oses/">this really well explained tuto about crontab</a>.

# You're done !

Any questions ? Any mistakes ? Tell me !